# DataFrames.jl And Julian Method Consistency
There are lots of things to love about the Julia programming language. Anyone that has become familiar with the language has likely heard the various talking points that always come up in discussions of why this language is so great. These topics range from Julia's speed to Julia's awesome type system. These are both great aspects of Julia that can certainly make a significant difference in the creation of great software. However, there are some other great features that the language boasts that are not discussed quite as often.

##### functions and their methods

In [1]:
function myfunc(x)

end

myfunc (generic function with 1 method)

In [2]:
methods(myfunc)

# 1 method for generic function "myfunc":
[1] myfunc(x) in Main at In[1]:1

In [3]:
any_method = methods(myfunc)[1]

myfunc(x) in Main at In[1]:1

In [4]:
any_method.sig

Tuple{typeof(myfunc), Any}

In [5]:
any_method.sig.parameters

svec(typeof(myfunc), Any)

In [6]:
any_method.sig.parameters[1]

typeof(myfunc) (singleton type of function myfunc, subtype of Function)

In [7]:
function myfunc(x::String)

end

myfunc (generic function with 2 methods)

In [8]:
methods(myfunc)

# 2 methods for generic function "myfunc":
[1] myfunc(x::String) in Main at In[7]:1
[2] myfunc(x) in Main at In[1]:1

In [9]:
import Base: string

In [10]:
struct Vec2{T <: Number}
    x::T
    y::T
end

In [11]:
string(v::Vec2{<:Any}) = "($(v.x), $(v.y))"

string (generic function with 21 methods)

In [12]:
string(Vec2(5, 5))
"(5, 5)"

"(5, 5)"

### important functions for data processing

In [13]:
x = [5, nothing, 4, nothing, 9, 8]

6-element Vector{Union{Nothing, Int64}}:
 5
  nothing
 4
  nothing
 9
 8

In [14]:
length(x)

6

In [15]:
isnothing(x[2])

true

In [16]:
x = [5, missing, 4, missing, 9, 8]

6-element Vector{Union{Missing, Int64}}:
 5
  missing
 4
  missing
 9
 8

In [17]:
ismissing(x[2])

true

In [18]:
filter!(x -> ~(ismissing(x)), x)

4-element Vector{Union{Missing, Int64}}:
 5
 4
 9
 8

In [19]:
findall(x -> x % 4 == 0, x)

2-element Vector{Int64}:
 2
 4

In [20]:
append!(x, 5)

5-element Vector{Union{Missing, Int64}}:
 5
 4
 9
 8
 5

In [21]:
x = [5, 10, 15]
y = [5, 10, 15]

3-element Vector{Int64}:
  5
 10
 15

In [22]:
v = Vector{Vector{Int64}}()

Vector{Int64}[]

In [23]:
push!(v, x, y)

2-element Vector{Vector{Int64}}:
 [5, 10, 15]
 [5, 10, 15]

In [24]:
v = Vector{Int64}()

Int64[]

In [25]:
append!(v, x, y)

6-element Vector{Int64}:
  5
 10
 15
  5
 10
 15

In [26]:
deleteat!(x, 2)

2-element Vector{Int64}:
  5
 15

In [27]:
insert!(x, 2, 5)

3-element Vector{Int64}:
  5
  5
 15

#### dimensions

In [28]:
x = [1 2; 1 2]

2×2 Matrix{Int64}:
 1  2
 1  2

In [29]:
x = hcat([1, 2], [1, 2])

2×2 Matrix{Int64}:
 1  1
 2  2

In [30]:
x = Matrix([1 2; 1 2])

2×2 Matrix{Int64}:
 1  2
 1  2

In [31]:
x = vcat(x, [5 6])

3×2 Matrix{Int64}:
 1  2
 1  2
 5  6

In [32]:
size(x)

(3, 2)

In [33]:
reshape([5, 6, 7, 8], 2, 2)

2×2 Matrix{Int64}:
 5  7
 6  8

In [34]:
reshape(x, 2, 3)

2×3 Matrix{Int64}:
 1  5  2
 1  2  6

In [35]:
transpose(x)

2×3 transpose(::Matrix{Int64}) with eltype Int64:
 1  1  5
 2  2  6

In [36]:
x

3×2 Matrix{Int64}:
 1  2
 1  2
 5  6

In [37]:
x  .* vcat([5 1], hcat([5, 6], [5, 6]))

3×2 Matrix{Int64}:
  5   2
  5  10
 30  36

In [38]:
using LinearAlgebra

In [39]:
LinearAlgebra.dot(x, hcat([5, 6], [4, 2], [3, 8]))

89

In [40]:
[5 6; 2 3] .* [5 6; 2 3; 4 3]

LoadError: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 2 and 3")

In [41]:
collect(eachrow(x))

3-element Vector{SubArray{Int64, 1, Matrix{Int64}, Tuple{Int64, Base.Slice{Base.OneTo{Int64}}}, true}}:
 [1, 2]
 [1, 2]
 [5, 6]

In [42]:
for x in eachrow(x)
    print(x, "   ")
end

[1, 2]   [1, 2]   [5, 6]   

In [43]:
collect(eachcol(x))

2-element Vector{SubArray{Int64, 1, Matrix{Int64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}}:
 [1, 1, 5]
 [2, 2, 6]

In [44]:
using DataFrames

df = DataFrame(:X => [1, 2, 3, 4, 5], :Y => [1, 2, 3, 4, 5])

[ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]


Row,X,Y
,Int64,Int64
1,1,1
2,2,2
3,3,3
4,4,4
5,5,5


In [45]:
filter!(r -> r[:X] > 1, df)

Row,X,Y
,Int64,Int64
1,2,2
2,3,3
3,4,4
4,5,5


In [46]:
push!(df, (5, 6))

Row,X,Y
,Int64,Int64
1,2,2
2,3,3
3,4,4
4,5,5
5,5,6


In [47]:
size(df)

(5, 2)

In [49]:
collect(eachcol(df))

2-element Vector{AbstractVector}:
 [2, 3, 4, 5, 5]
 [2, 3, 4, 5, 6]

In [50]:
collect(eachrow(df))

5-element Vector{DataFrameRow}:
 DataFrameRow
 Row │ X      Y     
     │ Int64  Int64 
─────┼──────────────
   1 │     2      2
 DataFrameRow
 Row │ X      Y     
     │ Int64  Int64 
─────┼──────────────
   2 │     3      3
 DataFrameRow
 Row │ X      Y     
     │ Int64  Int64 
─────┼──────────────
   3 │     4      4
 DataFrameRow
 Row │ X      Y     
     │ Int64  Int64 
─────┼──────────────
   4 │     5      5
 DataFrameRow
 Row │ X      Y     
     │ Int64  Int64 
─────┼──────────────
   5 │     5      6

In [52]:
dropmissing!(df)

Row,X,Y
,Int64,Int64
1,2,2
2,3,3
3,4,4
4,5,5
5,5,6


In [57]:
[5, 10, 15][Not(1, 2)]

1-element Vector{Int64}:
 15

In [56]:
df[!, Not(:X)]

Row,Y
,Int64
1,2
2,3
3,4
4,5
5,6
